In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
%matplotlib inline
import matplotlib.pyplot as plt
import re
import datetime as dt

from scipy import stats
from scipy.stats import pearsonr, spearmanr

from googletrans import Translator

from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE


import warnings
warnings.filterwarnings("ignore")

data = pd.read_csv('daily8_5.csv')
data['dateline'] = pd.to_datetime(data.dateline)
df = data.copy()
df = df[df.text_label != 'False']
df2 = df

In [2]:
df = data.copy()
df = df[df.text_label != 'False']
df2 = df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           237706 non-null  int64         
 1   id              237706 non-null  object        
 2   url             237706 non-null  object        
 3   headline        237706 non-null  object        
 4   dateline        237706 non-null  datetime64[ns]
 5   text            237706 non-null  object        
 6   tags            237706 non-null  object        
 7   source          237706 non-null  object        
 8   text_label      237706 non-null  object        
 9   text_score      237706 non-null  object        
 10  headline_label  237706 non-null  object        
 11  headline_score  237706 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 23.6+ MB


In [3]:
transcription_table=pd.DataFrame(
    {
        'source': ['Alittihad','Echoroukonline','Ryiadh','SaudiYoum','Techreen', 'Alqabas', 'Almustaqbal','Almasryalyoum', 'Youm7','Sabanews'],
        'country': ['UAE','Algeria','Saudi_Arabia','Saudi_Arabia','Syria', 'Kuwait', 'Lebanon','Egypt','Egypt','Yemen']
    }
)
mapping = transcription_table.set_index('source').to_dict()['country']
df['country'] = df['source'].apply(lambda x: mapping.get(x))

In [4]:
## Functions possibly required for this section

def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    'september_11th': pd.to_datetime('09-11-2001'),
    #'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    #'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    #'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['sept_11'] = df.dateline.apply(within_30_days, args = (date,))
    
    
def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    #'september_11th': pd.to_datetime('09-11-2001'),
    'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    #'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    #'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['capt_bag'] = df.dateline.apply(within_30_days, args = (date,))
    
    
    
    
def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    #'september_11th': pd.to_datetime('09-11-2001'),
    #'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    #'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['nick'] = df.dateline.apply(within_30_days, args = (date,))
    
    
    
def within_30_days(df_dateline, date):
    
    if (df_dateline - date).days < 30 and (df_dateline - date).days > -30:
        return 1
    else:
        return 0
    
important_dates = {
    #'september_11th': pd.to_datetime('09-11-2001'),
    #'capture_of_baghdad': pd.to_datetime('04-09-2003'),
    #'nick_berg': pd.to_datetime('05-12-2004'),
    #'iran_nulcear': pd.to_datetime('08-30-2006'),
    'arab_spring': pd.to_datetime('12-20-2011')
}

for event, date in important_dates.items():
    df['arab'] = df.dateline.apply(within_30_days, args = (date,))

In [5]:
topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}

def flip_key_value_pairs(dicts):
    
    res = dict((v,k) for k,v in dicts.items())    
    return res


def make_eng_tags(df_tags):
    eng_tags =[]
    
    rev_topics = flip_key_value_pairs(topics)
    for key, value in rev_topics.items():
        if key in df_tags:
            eng_tags.append(value)
            
    return eng_tags

df['english_tags'] = df.tags.apply(make_eng_tags)
#translates tags to english

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           237706 non-null  int64         
 1   id              237706 non-null  object        
 2   url             237706 non-null  object        
 3   headline        237706 non-null  object        
 4   dateline        237706 non-null  datetime64[ns]
 5   text            237706 non-null  object        
 6   tags            237706 non-null  object        
 7   source          237706 non-null  object        
 8   text_label      237706 non-null  object        
 9   text_score      237706 non-null  object        
 10  headline_label  237706 non-null  object        
 11  headline_score  237706 non-null  float64       
 12  country         237706 non-null  object        
 13  sept_11         237706 non-null  int64         
 14  capt_bag        237706 non-null  int

In [7]:
df['english_tags'] = df.english_tags.astype('str')

In [8]:
df["quarter"] = pd.PeriodIndex(df.dateline, freq='Q')
df["year"] = pd.PeriodIndex(df.dateline, freq='Y')
df["half_year"] = pd.PeriodIndex(df.dateline, freq='6M')

In [9]:
df['year'] = df['year'].astype('str')
df['quarter'] = df['quarter'].astype('str')
df['half_year'] = df['half_year'].astype('str')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   index           237706 non-null  int64         
 1   id              237706 non-null  object        
 2   url             237706 non-null  object        
 3   headline        237706 non-null  object        
 4   dateline        237706 non-null  datetime64[ns]
 5   text            237706 non-null  object        
 6   tags            237706 non-null  object        
 7   source          237706 non-null  object        
 8   text_label      237706 non-null  object        
 9   text_score      237706 non-null  object        
 10  headline_label  237706 non-null  object        
 11  headline_score  237706 non-null  float64       
 12  country         237706 non-null  object        
 13  sept_11         237706 non-null  int64         
 14  capt_bag        237706 non-null  int

In [11]:
x = pd.get_dummies(df[['source', 'country', 'quarter', 'year', 
                    'half_year']])

In [12]:
def encode_tags(df):
    list_of_tags = []
    for tag in df.english_tags.values:
        list_of_tags.extend([val[1:-1] for val in tag[1:-1].split(', ')])
    tag_list = list(set(list_of_tags))
    for tag in tag_list:
        df[tag] = 0
    for i, tag in enumerate(df.english_tags):
        for t in tag_list:
            if t in tag:
                df[t].iloc[i] = 1
    return df

In [13]:
encode_tags(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 39 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   index               237706 non-null  int64         
 1   id                  237706 non-null  object        
 2   url                 237706 non-null  object        
 3   headline            237706 non-null  object        
 4   dateline            237706 non-null  datetime64[ns]
 5   text                237706 non-null  object        
 6   tags                237706 non-null  object        
 7   source              237706 non-null  object        
 8   text_label          237706 non-null  object        
 9   text_score          237706 non-null  object        
 10  headline_label      237706 non-null  object        
 11  headline_score      237706 non-null  float64       
 12  country             237706 non-null  object        
 13  sept_11             237706 no

In [15]:
#All columns that need to be dropped
x2 = df.drop(columns=['index', 'id', 'url', 'headline', 'dateline', 'text', 'english_tags', 'tags', 'text_label', 'text_score', 'headline_label', 'headline_score', 'source', 'country', 'quarter', 'year', 
                    'half_year'])

In [16]:
x3 = pd.concat([x, x2], axis=1)

In [17]:
x3

,source_Alittihad,source_Almasryalyoum,source_Almustaqbal,source_Alqabas,source_Echoroukonline,source_Ryiadh,source_Sabanews,source_SaudiYoum,source_Techreen,source_Youm7,...,Clinton,America,Gulf War,Google,World Trade Center,American (pl),Cheney,Osama Bin Laden,Bush,September 11
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331864,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331865,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
331866,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331867,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
df['text_label'] = df['text_label'].replace('neutral', 0)
df['text_label'] = df['text_label'].replace('positive', 1)
df['text_label'] = df['text_label'].replace('negative', -1)
df['text_label'] = df['text_label'].astype(int)
y = df['text_label']

In [19]:
f_selector = SelectKBest(f_regression, k=8)

# find the top 8 X's correlated with y
f_selector.fit(x3, y)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = x3.iloc[:,feature_mask].columns.tolist()
f_feature

['source_Alittihad',
 'source_Almasryalyoum',
 'source_Techreen',
 'source_Youm7',
 'country_Syria',
 'country_UAE',
 'America',
 'Bush']

In [20]:
scores = f_selector.scores_


# get the ranks
var_ranks = f_selector.scores_
# get the variable names
var_names = x3.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Scores': var_ranks})
# sort the df by rank
scores = rfe_ranks_df.sort_values('Scores')

In [21]:
def ramadan(df):
    date_list = []
    for i in range(len(df)):
        if (
        (df.dateline[i] >=dt.datetime(2001, 11, 17)) & (df.dateline[i] <= dt.datetime(2001, 12, 16))
        or (df.dateline[i] >=dt.datetime(2002, 11, 6)) & (df.dateline[i] <= dt.datetime(2002, 12, 5))
        or (df.dateline[i] >=dt.datetime(2003, 10, 27)) & (df.dateline[i] <= dt.datetime(2003, 11, 25))
        or (df.dateline[i] >=dt.datetime(2004, 10, 16)) & (df.dateline[i] <= dt.datetime(2004, 11, 13))
        or (df.dateline[i] >=dt.datetime(2005, 10, 5)) & (df.dateline[i] <= dt.datetime(2005, 11, 2))
        or (df.dateline[i] >=dt.datetime(2006, 9, 24)) & (df.dateline[i] <= dt.datetime(2006, 10, 23))
        or (df.dateline[i] >=dt.datetime(2007, 9, 13)) & (df.dateline[i] <= dt.datetime(2007, 10, 12))
        or (df.dateline[i] >=dt.datetime(2008, 9, 2)) & (df.dateline[i] <= dt.datetime(2008, 10, 1))
        or (df.dateline[i] >=dt.datetime(2009, 8, 22)) & (df.dateline[i] <= dt.datetime(2009, 9, 20))
        or (df.dateline[i] >=dt.datetime(2010, 8, 11)) & (df.dateline[i] <= dt.datetime(2010, 9, 9))
        or (df.dateline[i] >=dt.datetime(2011, 8, 1)) & (df.dateline[i] <= dt.datetime(2011, 8, 30))
        or (df.dateline[i] >=dt.datetime(2012, 7, 20)) & (df.dateline[i] <= dt.datetime(2012, 8, 18))
        or (df.dateline[i] >=dt.datetime(2013, 7, 9)) & (df.dateline[i] <= dt.datetime(2013, 8, 7))
        or (df.dateline[i] >=dt.datetime(2014, 6, 29)) & (df.dateline[i] <= dt.datetime(2014, 7, 28))):
            date_list.append(1)
        else:
            date_list.append(0)
    return date_list

In [29]:
df = df.reset_index()

In [30]:
df['is_ramadan'] = ramadan(df)

In [24]:
def is_gov_controlled(entry):
    if entry in ['Alqabas', 'Echoroukonline', 'Ryiadh', 'Saudiyoum', 'Almustaqbal', 'Youm7', 'Almasryalyoum']:
        return 1
    else:
        return 0

df['ownership_status'] = df.source.apply(is_gov_controlled)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237706 entries, 0 to 237705
Data columns (total 45 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   level_0                   237706 non-null  int64         
 1   index                     237706 non-null  int64         
 2   id                        237706 non-null  object        
 3   url                       237706 non-null  object        
 4   headline                  237706 non-null  object        
 5   dateline                  237706 non-null  datetime64[ns]
 6   text                      237706 non-null  object        
 7   tags                      237706 non-null  object        
 8   source                    237706 non-null  object        
 9   text_label                237706 non-null  int64         
 10  text_score                237706 non-null  object        
 11  headline_label            237706 non-null  object        
 12  he

In [26]:
transcription_table=pd.DataFrame(
    {
        'source': ['Alittihad','Echoroukonline','Ryiadh','SaudiYoum','Techreen', 'Alqabas', 'Almustaqbal','Almasryalyoum', 'Youm7','Sabanews'],
        'year': [ 1969 , 1991 , 1965 , 1965 , 1975 ,  1972 ,  1999 , 2004 , 2008 , 1990]
    }
)
mapping = transcription_table.set_index('source').to_dict()['year']
df['source_start_year'] = df['source'].apply(lambda x: mapping.get(x))


df['scaled_date'] = (df.dateline.astype('datetime64') - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(df[['scaled_date', 'source_start_year']])

df[['scaled_date', 'source_start_year_scaled']] = scaler.transform(df[['scaled_date', 'source_start_year']])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 237706 entries, 1 to 331869
Data columns (total 43 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   index                     237706 non-null  int64         
 1   id                        237706 non-null  object        
 2   url                       237706 non-null  object        
 3   headline                  237706 non-null  object        
 4   dateline                  237706 non-null  datetime64[ns]
 5   text                      237706 non-null  object        
 6   tags                      237706 non-null  object        
 7   source                    237706 non-null  object        
 8   text_label                237706 non-null  int64         
 9   text_score                237706 non-null  object        
 10  headline_label            237706 non-null  object        
 11  headline_score            237706 non-null  float64       
 12  co

In [37]:
scores.sort_values(by='Scores', ascending=False).tail(30)

,Var,Scores
213,half_year_2011-10,0.645154
106,half_year_2002-11,0.487768
117,half_year_2003-10,0.481563
108,half_year_2003-01,0.431334
204,half_year_2011-01,0.418717
240,half_year_2014-01,0.408919
148,half_year_2006-05,0.397086
40,quarter_2006Q2,0.307103
262,World Trade Center,0.220662
31,quarter_2004Q1,0.208068
